# Wikification with OpenAI: Leveraging Language Models for Entity Linking and Annotation

[Wikification](https://en.wikipedia.org/wiki/Wikification) is a fundamental task in [natural language processing](https://en.wikipedia.org/wiki/Natural_language_processing) that involves linking named entities in a given text to their corresponding entries in a knowledge base, such as Wikipedia. In this notebook, we will explore how to leverage [OpenAI's](https://en.wikipedia.org/wiki/OpenAI) powerful language models to perform wikification using the [LLM model](https://en.wikipedia.org/wiki/Language_model).
Entity linking and annotation are crucial for a wide range of applications, including [information retrieval](https://en.wikipedia.org/wiki/Information_retrieval), [question answering systems](https://en.wikipedia.org/wiki/Question_answering), [text summarization](https://en.wikipedia.org/wiki/Text_summarization), and more. By linking entities to knowledge bases, we can enrich the understanding of text and facilitate further analysis.
Throughout this notebook, we will guide you through the process of setting up the environment, installing the necessary libraries, and implementing the steps involved in performing wikification with [OpenAI's](https://en.wikipedia.org/wiki/OpenAI) [LLM model](https://en.wikipedia.org/wiki/Language_model).

In [325]:
from dotenv import load_dotenv
from pathlib import Path
import json
import openai 
import os


In [326]:
env_path = Path('../../../.env.sample') # Change with your .env file
load_dotenv(dotenv_path=env_path,override=True) # Use a text-davinci-003 model

True

In [327]:
openai.api_type = "azure"
openai.api_key = os.getenv('OPENAI_API_KEY') 
openai.api_base = os.getenv("OPENAI_API_BASE")

COMPLETIONS_MODEL = os.getenv("COMPLETIONS_MODEL") 

In [329]:
def wikifi_text(text,completions_model,max_tokens=300):
    prompt =  """You will read a text, and you will detect named entities contained in it 
--- 
TEXT """+text+"""  
---
Now you will connect the named entities to a corresponding Wikipedia Page. 

If there are several wikipedia pages corresponding, select wikipedia pages that have a logic connection

e.g. If in the same sentence I have Italy and Roberto Baggio, Italy is probably referring to the National Soccer Team

Finally you will return the result in a json format like the one below:

{
    "annotations" : [
      { 
        "wikipedia_page":"HERE PUT THE WIKIPEDIA PAGE URL",
        "wikipedia_page_title":"HERE PUT THE WIKIPEDIA PAGE TITLE",
        "text":"HERE PUT THE DETECTED TEXT"
      },
       ... ]
}

Answer:
"""
    result = openai.Completion.create(
        prompt=prompt,
        temperature=0.5,
        max_tokens=max_tokens,
        engine=completions_model
    )
    return json.loads(result['choices'][0]['text'].strip("\n").replace("\n",""))


In [330]:
def print_wikification_result(wikification):
    print(wikification)
    for (index,ann) in enumerate(wikification['annotations']):
        print(f"#{index}")
        print(" - "+ann["text"])
        print(" - "+ann["wikipedia_page_title"])
        print(" - "+ann["wikipedia_page"])
        print("\n")




In [331]:
sample_text_1 = "On this day 24 years ago Maradona scored his infamous Hand of God goal against England in the quarter-final of the 1986"
sample_text_2 = "The Mona Lisa is a sixteenth century oil painting created by Leonardo. It's held at the Louvre in Paris."

In [332]:
wikification_test_1 = wikifi_text(sample_text_1,completions_model=COMPLETIONS_MODEL)
print_wikification_result(wikification_test_1)

{'annotations': [{'wikipedia_page': 'https://en.wikipedia.org/wiki/Diego_Maradona', 'wikipedia_page_title': 'Diego Maradona', 'text': 'Maradona'}, {'wikipedia_page': 'https://en.wikipedia.org/wiki/1986_FIFA_World_Cup', 'wikipedia_page_title': '1986 FIFA World Cup', 'text': '1986'}, {'wikipedia_page': 'https://en.wikipedia.org/wiki/England_national_football_team', 'wikipedia_page_title': 'England national football team', 'text': 'England'}]}
#0
 - Maradona
 - Diego Maradona
 - https://en.wikipedia.org/wiki/Diego_Maradona


#1
 - 1986
 - 1986 FIFA World Cup
 - https://en.wikipedia.org/wiki/1986_FIFA_World_Cup


#2
 - England
 - England national football team
 - https://en.wikipedia.org/wiki/England_national_football_team




In [333]:
wikification_test_2 = wikifi_text(sample_text_2,completions_model=COMPLETIONS_MODEL)
print_wikification_result(wikification_test_2)

{'annotations': [{'wikipedia_page': 'https://en.wikipedia.org/wiki/Mona_Lisa', 'wikipedia_page_title': 'Mona Lisa', 'text': 'Mona Lisa'}, {'wikipedia_page': 'https://en.wikipedia.org/wiki/Leonardo_da_Vinci', 'wikipedia_page_title': 'Leonardo da Vinci', 'text': 'Leonardo'}, {'wikipedia_page': 'https://en.wikipedia.org/wiki/Louvre', 'wikipedia_page_title': 'Louvre', 'text': 'Louvre'}, {'wikipedia_page': 'https://en.wikipedia.org/wiki/Paris', 'wikipedia_page_title': 'Paris', 'text': 'Paris'}]}
#0
 - Mona Lisa
 - Mona Lisa
 - https://en.wikipedia.org/wiki/Mona_Lisa


#1
 - Leonardo
 - Leonardo da Vinci
 - https://en.wikipedia.org/wiki/Leonardo_da_Vinci


#2
 - Louvre
 - Louvre
 - https://en.wikipedia.org/wiki/Louvre


#3
 - Paris
 - Paris
 - https://en.wikipedia.org/wiki/Paris




In case you're wondering... yes! The introduction of this notebook was wikified using the same approach! Let's try!

In [334]:
sample_text_3 = """Wikification is a fundamental task in natural language processing that involves linking named entities in a given text to their corresponding entries in a knowledge base, such as Wikipedia. In this notebook, we will explore how to leverage OpenAI's powerful language models to perform wikification using the LLM model.
Entity linking and annotation are crucial for a wide range of applications, including information retrieval, question answering systems, text summarization, and more. By linking entities to knowledge bases, we can enrich the understanding of text and facilitate further analysis.
Throughout this notebook, we will guide you through the process of setting up the environment, installing the necessary libraries, and implementing the steps involved in performing wikification with OpenAI's LLM model."""

In [335]:
wikification_test_3 = wikifi_text(sample_text_3,completions_model=COMPLETIONS_MODEL,max_tokens=1000)
print_wikification_result(wikification_test_3)

{'annotations': [{'wikipedia_page': 'https://en.wikipedia.org/wiki/Natural_language_processing', 'wikipedia_page_title': 'Natural language processing', 'text': 'natural language processing'}, {'wikipedia_page': 'https://en.wikipedia.org/wiki/Information_retrieval', 'wikipedia_page_title': 'Information retrieval', 'text': 'information retrieval'}, {'wikipedia_page': 'https://en.wikipedia.org/wiki/Question_answering', 'wikipedia_page_title': 'Question answering', 'text': 'question answering systems'}, {'wikipedia_page': 'https://en.wikipedia.org/wiki/Text_summarization', 'wikipedia_page_title': 'Text summarization', 'text': 'text summarization'}, {'wikipedia_page': 'https://en.wikipedia.org/wiki/OpenAI', 'wikipedia_page_title': 'OpenAI', 'text': "OpenAI's"}, {'wikipedia_page': 'https://en.wikipedia.org/wiki/Language_model', 'wikipedia_page_title': 'Language model', 'text': 'LLM model'}]}
#0
 - natural language processing
 - Natural language processing
 - https://en.wikipedia.org/wiki/Nat

In [336]:
sample_text_3_wikified = sample_text_3
for ann in wikification_test_3['annotations']:
    markdown_text= "["+ann["text"]+"]("+ann["wikipedia_page"]+")"
    sample_text_3_wikified = sample_text_3_wikified.replace(ann["text"],markdown_text)
    
print("Final Markdown text:\n\n"+sample_text_3_wikified)

Final Markdown text:

Wikification is a fundamental task in [natural language processing](https://en.wikipedia.org/wiki/Natural_language_processing) that involves linking named entities in a given text to their corresponding entries in a knowledge base, such as Wikipedia. In this notebook, we will explore how to leverage [OpenAI's](https://en.wikipedia.org/wiki/OpenAI) powerful language models to perform wikification using the [LLM model](https://en.wikipedia.org/wiki/Language_model).
Entity linking and annotation are crucial for a wide range of applications, including [information retrieval](https://en.wikipedia.org/wiki/Information_retrieval), [question answering systems](https://en.wikipedia.org/wiki/Question_answering), [text summarization](https://en.wikipedia.org/wiki/Text_summarization), and more. By linking entities to knowledge bases, we can enrich the understanding of text and facilitate further analysis.
Throughout this notebook, we will guide you through the process of sett